In [1]:
# we install all the required for RelBench
!pip install relbench[full]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.2 MB/s eta 0:00:00


In [2]:
import relbench
from relbench.datasets import get_dataset_names, get_dataset
from relbench.modeling.utils import get_stype_proposal
from relbench.modeling.graph import make_pkey_fkey_graph


import torch
from torch_geometric.seed import seed_everything
from torch import Tensor
from torch_frame.config.text_embedder import TextEmbedderConfig


from typing import List, Optional


from sentence_transformers import SentenceTransformer


import os


import pandas as pd


import random

import pickle

import requests

In [3]:
class GloveTextEmbedding:
    def __init__(self, device: Optional[torch.device] = None):
        self.model = SentenceTransformer(
            "sentence-transformers/average_word_embeddings_glove.6B.300d",
            device=device,
        )

    def __call__(self, sentences: List[str]) -> Tensor:
        return self.model.encode(sentences, convert_to_tensor=True)

# Environment Checking

In [4]:
# Check that it's cuda if you want it to run in reasonable time!
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    torch.set_num_threads(1)
print(device)

# Set the seed for generating random numbers to ensure reproducibility
seed_everything(42)

# Path to the directory for caching graph data
root_dir = "./data"

# Configure the text encoder
text_embedder_cfg = TextEmbedderConfig(
    text_embedder=GloveTextEmbedding(device=device),
    batch_size=256
)

cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/248 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

(…)WordEmbeddings/wordembedding_config.json:   0%|          | 0.00/164 [00:00<?, ?B/s]

(…)beddings/whitespacetokenizer_config.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/480M [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
print(f"The RelBench version is {relbench.__version__}")
print(f"The RelBench datasets are {get_dataset_names()}")

The RelBench version is 1.1.0
The RelBench datasets are ['rel-amazon', 'rel-avito', 'rel-event', 'rel-f1', 'rel-hm', 'rel-stack', 'rel-trial']


#Usefull functions

In [ ]:
def modify_node_name(key):
    # Trova l'ultima posizione di '_' e rimuove la parte numerica
    last_underscore_index = key.rfind('_')

    if last_underscore_index != -1:
        # Parte letterale (es. 'standings')
        modified_name = key[:last_underscore_index]

        # Parte numerica (es. 3)
        node_index = int(''.join(filter(str.isdigit, key[last_underscore_index:])))

        return modified_name, node_index

    else:
        node_name = ''.join(filter(str.isalpha, key))  # Parte letterale (es. 'standings')
        node_index = int(''.join(filter(str.isdigit, key)))  # Parte numerica (es. 5)
        return node_name, node_index

In [ ]:
def read_triplets_from_file(file_path):
    triplets = []

    # Controlla se il file_path è un URL
    if file_path.startswith("http://") or file_path.startswith("https://"):
        response = requests.get(file_path)
        if response.status_code == 200:
            lines = response.text.splitlines()
        else:
            print(f"Errore nel download del file: {response.status_code}")
            return triplets
    else:
        # Legge il file locale
        with open(file_path, 'r') as file:
            lines = file.readlines()

    for line in lines:
        # Rimuovi eventuali spazi bianchi e separa la riga in base ai tab
        parts = line.strip().split('\t')
        if len(parts) == 3:  # Assicurati che ci siano esattamente 3 elementi
            triplet = (parts[0], parts[1], parts[2])
            triplets.append(triplet)
        else:
            print(f"Riga non valida: {line.strip()}")

    return triplets


In [ ]:
def build_nodes_dictionary_from_triplets(node_dict, triplets, nodes_without_timestamp, split):

    for triplet in triplets:
      source_node = triplet[0]
      edge_label = triplet[1]
      target_node = triplet[2]

      # se il source node e il target node non sono già nel vocabolario li aggiungo
      if source_node not in node_dict:
          source_node_label = modify_node_name(source_node)

          if source_node_label in nodes_without_timestamp:
            node_dict[source_node] = [len(node_dict), 'all']
          else:
            node_dict[source_node] = [len(node_dict), split]

      if target_node not in node_dict:
          target_node_label = modify_node_name(target_node)

          if target_node_label in nodes_without_timestamp:
            node_dict[target_node] = [len(node_dict), 'all']
          else:
            node_dict[target_node] = [len(node_dict), split]

    return node_dict

In [ ]:
def build_GraphAny_dataset(KG_data, node_dict, triplets):
    node_features = []
    labels = []
    edges = []

    train_mask = []
    val_mask = []
    test_mask = []

    for key, value in node_dict.items():
      # dalla chiave ottengo il nome del nodo e l'indice
      entity_label, entity_index = modify_node_name(key)

      # da KG_data prendo le features di quell'entità e la aggiungo a node_features
      node_features.append(KG_data[entity_label].tf[entity_index])

      # aggiugo la label di quell'entità a labels
      labels.append(entity_label)

      # in base al valore di split aggiorno le maschere
      split_string = value[1]
      if split_string == "train":
        train_mask.append(True)
        val_mask.append(False)
        test_mask.append(False)
      elif split_string == "val":
        train_mask.append(False)
        val_mask.append(True)
        test_mask.append(False)
      elif split_string == "test":
        train_mask.append(False)
        val_mask.append(False)
        test_mask.append(True)
      elif split_string == 'all':
        train_mask.append(True)
        val_mask.append(True)
        test_mask.append(True)


    # per ogni tripletta passata ricavo source e target
    for triplet in triplets:
      source_node = triplet[0]
      edge_label = triplet[1]
      target_node = triplet[2]

      source_index = node_dict[source_node][0]
      target_index = node_dict[target_node][0]

      pair = [source_index, target_index]
      if pair not in edges:
        edges.append(pair)

    return node_features, labels, edges, train_mask, val_mask, test_mask

# F1 Dataset Creation

In [6]:
# We download the f1-dataset
f1_dataset = get_dataset(name="rel-f1", download=True)

# we download the entire database (also the test part)
f1_db = f1_dataset.get_db(upto_test_timestamp = False)
f1_col_to_stype_dict = get_stype_proposal(f1_db)

# Generate graph data
f1_data, f1_col_stats_dict = make_pkey_fkey_graph(
    f1_db,
    col_to_stype_dict = f1_col_to_stype_dict,  # Column types
    text_embedder_cfg = text_embedder_cfg,  # Our chosen text encoder
    cache_dir=os.path.join(
        root_dir, f"rel-f1_materialized_cache"
    ),  # Store materialized graph for convenience
)

100%|████████████████████████████████████████| 704k/704k [00:00<00:00, 586MB/s]
Unzipping contents of '/root/.cache/relbench/rel-f1/db.zip' to '/root/.cache/relbench/rel-f1/.'


Loading Database object from /root/.cache/relbench/rel-f1/db...
Done in 0.15 seconds.


Embedding raw data in mini-batch: 100%|██████████| 1/1 [00:00<00:00, 258.49it/s]
/usr/local/lib/python3.11/dist-packages/torch_frame/data/stats.py:177: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ser = pd.to_datetime(ser, format=time_format)
Embedding raw data in mini-batch: 100%|██████████| 5/5 [00:00<00:00, 221.62it/s]
/usr/local/lib/python3.11/dist-packages/torch_frame/data/mapper.py:291: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ser = pd.to_datetime(ser, format=self.format, errors='coerce')
Embedding raw data in mini-batch: 100%|██████████| 1/1 [00:00<00:00, 146.14it/s]


In [7]:
f1_val_timestep = f1_dataset.val_timestamp
f1_test_timestep = f1_dataset.test_timestamp

print(f"The validation timestep is: {f1_val_timestep}")
print(f"The test timestep is: {f1_test_timestep}")

The validation timestep is: 2005-01-01 00:00:00
The test timestep is: 2010-01-01 00:00:00


In [ ]:
f1_data

HeteroData(
  drivers={ tf=TensorFrame([857, 6]) },
  results={
    tf=TensorFrame([26080, 11]),
    time=[26080],
  },
  standings={
    tf=TensorFrame([34124, 4]),
    time=[34124],
  },
  constructor_results={
    tf=TensorFrame([12290, 2]),
    time=[12290],
  },
  constructors={ tf=TensorFrame([211, 3]) },
  circuits={ tf=TensorFrame([77, 7]) },
  qualifying={
    tf=TensorFrame([9815, 3]),
    time=[9815],
  },
  constructor_standings={
    tf=TensorFrame([13051, 4]),
    time=[13051],
  },
  races={
    tf=TensorFrame([1101, 5]),
    time=[1101],
  },
  (results, f2p_raceId, races)={ edge_index=[2, 26080] },
  (races, rev_f2p_raceId, results)={ edge_index=[2, 26080] },
  (results, f2p_driverId, drivers)={ edge_index=[2, 26080] },
  (drivers, rev_f2p_driverId, results)={ edge_index=[2, 26080] },
  (results, f2p_constructorId, constructors)={ edge_index=[2, 26080] },
  (constructors, rev_f2p_constructorId, results)={ edge_index=[2, 26080] },
  (standings, f2p_raceId, races)={ edge

In [ ]:
node_names = ['standings', 'drivers', 'results', 'constructor_results', 'circuits', 'qualifying', 'races', 'constructors', 'constructor_standings']
node_without_timestamp = ['drivers', 'circuits', 'constructors']
edges_names = [('constructor_standings', 'f2p_raceId', 'races'),
                ('races', 'rev_f2p_raceId', 'constructor_standings'),
                ('constructor_standings', 'f2p_constructorId', 'constructors'),
                ('constructors', 'rev_f2p_constructorId', 'constructor_standings'),
                ('standings', 'f2p_raceId', 'races'),
                ('races', 'rev_f2p_raceId', 'standings'),
                ('standings', 'f2p_driverId', 'drivers'),
                ('drivers', 'rev_f2p_driverId', 'standings'),
                ('constructor_results', 'f2p_raceId', 'races'),
                ('races', 'rev_f2p_raceId', 'constructor_results'),
                ('constructor_results', 'f2p_constructorId', 'constructors'),
                ('constructors', 'rev_f2p_constructorId', 'constructor_results'),
                ('results', 'f2p_raceId', 'races'),
                ('races', 'rev_f2p_raceId', 'results'),
                ('results', 'f2p_driverId', 'drivers'),
                ('drivers', 'rev_f2p_driverId', 'results'),
                ('results', 'f2p_constructorId', 'constructors'),
                ('constructors', 'rev_f2p_constructorId', 'results'),
                ('qualifying', 'f2p_raceId', 'races'),
                ('races', 'rev_f2p_raceId', 'qualifying'),
                ('qualifying', 'f2p_driverId', 'drivers'),
                ('drivers', 'rev_f2p_driverId', 'qualifying'),
                ('qualifying', 'f2p_constructorId', 'constructors'),
                ('constructors', 'rev_f2p_constructorId', 'qualifying'),
                ('races', 'f2p_circuitId', 'circuits'),
                ('circuits', 'rev_f2p_circuitId', 'races')]

In [ ]:
train_path = "https://raw.githubusercontent.com/RiccardoRomeo01/BDATM_project_public_data/main/datasets/F1-v2/inductive/train.txt"
val_path = "https://raw.githubusercontent.com/RiccardoRomeo01/BDATM_project_public_data/main/datasets/F1-v2/inductive/inference_valid.txt"
test_path = "https://raw.githubusercontent.com/RiccardoRomeo01/BDATM_project_public_data/main/datasets/F1-v2/inductive/inference_test.txt"

In [ ]:
train_triplets = read_triplets_from_file(train_path)
val_triplets = read_triplets_from_file(val_path)
test_triplets = read_triplets_from_file(test_path)

In [ ]:
f1_dict = {}
f1_dict = build_nodes_dictionary_from_triplets(node_dict = f1_dict,
                                               triplets = train_triplets,
                                               nodes_without_timestamp = node_without_timestamp,
                                               split = 'train')
f1_dict = build_nodes_dictionary_from_triplets(node_dict = f1_dict,
                                               triplets = val_triplets,
                                               nodes_without_timestamp = node_without_timestamp,
                                               split = 'val')
f1_dict = build_nodes_dictionary_from_triplets(node_dict = f1_dict,
                                               triplets = test_triplets,
                                               nodes_without_timestamp = node_without_timestamp,
                                               split = 'test')

In [ ]:
f1_triplets = train_triplets + val_triplets + test_triplets

In [ ]:
f1_node_features, f1_labels, f1_edges, f1_train_mask, f1_val_mask, f1_test_mask = build_GraphAny_dataset(KG_data = f1_data,
                                                                                                          node_dict = f1_dict,
                                                                                                          triplets = f1_triplets)

In [ ]:
data_to_save = {
    'node_features': f1_node_features,
    'labels': f1_labels,
    'edges': f1_edges,
    'train_mask': f1_train_mask,
    'val_mask': f1_val_mask,
    'test_mask': f1_test_mask
}

with open('f1_data.pkl', 'wb') as f:
    pickle.dump(data_to_save, f)

In [8]:
# PER LEGGERE DAL FILE
import requests
from torch_frame import stype
from torch_frame.nn import (
    EmbeddingEncoder,
    LinearEmbeddingEncoder,
    LinearEncoder,
    TimestampEncoder,
    StypeWiseFeatureEncoder,
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


file_url = "https://raw.githubusercontent.com/RiccardoRomeo01/BDATM_project_public_data/main/GraphAny_datasets/f1_9_classes.pkl"

# Scarica il file
response = requests.get(file_url)
response.raise_for_status()  # Verifica che la richiesta sia andata a buon fine

# Carica i dati dal contenuto del file
data_loaded = pickle.loads(response.content)

f1_node_features = data_loaded['node_features']
f1_labels = data_loaded['labels']
f1_edges = data_loaded['edges']
f1_train_mask = data_loaded['train_mask']
f1_val_mask = data_loaded['val_mask']
f1_test_mask = data_loaded['test_mask']

In [13]:
print(f"La variabile f1_node_features è una lista di TensorFrame contenenti ognuno le feature di un nodo del nostro grafo. Quindi abbiamo in totale {len(f1_node_features)} nodi nel nostro grafo.\n")

print(f"La variabile f1_labels è una lista di stringhe che rappresentano il tipo di nodo (la sua label). Per esempio, il primo nodo del nostro grafo è un nodo di tipo: {f1_labels[0]}\n")

print(f"La variabile f1_edges è una lista di tuple che rappresentano le coppie di nodi connessi da un arco. Quindi abbiamo in totale {len(f1_edges)} archi nel nostro grafo.\n")

print(f"Le variabili 'mask' rappresentano le maschere. Si tratta di liste di booleani che indicano quali nodi fanno parte di quali set.")

La variabile f1_node_features è una lista di TensorFrame contenenti ognuno le feature di un nodo del nostro grafo. Quindi abbiamo in totale 12553 nodi nel nostro grafo.

La variabile f1_labels è una lista di stringhe che rappresentano il tipo di nodo (la sua label). Per esempio, il primo nodo del nostro grafo è un nodo di tipo: results

La variabile f1_edges è una lista di tuple che rappresentano le coppie di nodi connessi da un arco. Quindi abbiamo in totale 11362 archi nel nostro grafo.

Le variabili 'mask' rappresentano le maschere. Si tratta di liste di booleani che indicano quali nodi fanno parte di quali set.


In [15]:
stype_encoder_dict = {
    stype.categorical: EmbeddingEncoder(),
    stype.numerical: LinearEncoder(),
    stype.embedding: LinearEmbeddingEncoder(),
    stype.timestamp: TimestampEncoder()
}



f1_node_features_tensors = []
for i, tensor_frame in enumerate(f1_node_features):
    featureEncoder = StypeWiseFeatureEncoder(
                                    out_channels = tensor_frame.num_cols,
                                    col_stats = f1_col_stats_dict[f1_labels[i]],  # usa l'indice di f1_labels
                                    col_names_dict = tensor_frame.col_names_dict,
                                    stype_encoder_dict = stype_encoder_dict
                                ).to(device)
    tensor = featureEncoder(tensor_frame.to(device))
    f1_node_features_tensors.append(tensor)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x0 and 300x3)

# Comprendiamo cosa diamo in pasto a StypeWiseFeatureEncoder

Qui faccio vedere cosa in teoria diamo in pasto a StypeWiseFeatureEncoder, lo farò mostrando un solo elemento del ciclo e spiegando ogni passaggio.

In [18]:
# Noi vogliamo trasformare tutte le feature da formati TensorFrame a normali tensori. Per farlo dobbiamo usare un StypeWiseFeatureEncoder.
# Però, ogni nodo del grafo è  diverso, quindi creiamo un StypeWiseFeatureEncoder differente per ogni nodo, in modo da dare in pasto a StypeWiseFeatureEncoder i parametri giusti.
for i, tensor_frame in enumerate(f1_node_features):
  # Qui inserisco un if che normalmente non metterei, lo faccio solo per evidenziare il problema che abbiamo.
  # In particolare, i dati categorici sono un problema. Quindi, al primo dato di tipo categorico fermo il ciclo
  # Per ottenere i tipi dei dati (stype) uso col_names_dict del TensorFrame e ne seleziono le chiavi
  if stype.categorical in tensor_frame.col_names_dict.keys():

    print(f"Il TensorFrame incontrato ha un elemento di tipo categorico al suo interno: \n{tensor_frame}\n\n")

    #### out_channels ####
    '''
      Il primo valore da dare in pasto è out_channels, che rappresenta la dimensione di uscita del tensore.
      Ogni TensorFrame ha un parametro chiamato num_cols  che ho deciso di utilizzare come canale di output: out_channels = tensor_frame.num_cols
    '''
    print(f"Il numero di colonne di questo TensorFrame è: {tensor_frame.num_cols}, quindi out_channels sarà uguale a {tensor_frame.num_cols}\n\n")

    #### col_stats ####
    '''
      col_stats è un dizionario che contiene le statistiche per ogni colonna, ma sono le statistiche delle colonne di quello specifico TensorFrame.
      Per ottenere questi valori uso la variabile f1_col_stats_dict ottenuta tramite RelBench. Si tratta di un dizionario con tutte le statistiche delle colonne di ogni TensorFrame nel nostro grafo.
      Quindi avremo che col_stats = f1_col_stats_dict[f1_labels[i]]. Nota che ho usato f1_labels[i] per selezionare il giusto nome del TensorFrame.
      Infatti se l'i-esimo TensorFrame è di tipo 'standings' allora col_stats = f1_col_stats_dict['standings'].
    '''
    print(f"Questo TensorFrame è di tipo: {f1_labels[i]}")
    print(f"Le statistiche di questo TensorFrame sono: \n{f1_col_stats_dict[f1_labels[i]]}\n\n")

    #### col_names_dict ####
    '''
      Il terzo parametro da passare dentro StypeWiseFeatureEncoder è col_names_dict. Si tratta di un dizionario che indica il tipo di dato stype per ogni colonna del TensorFrame
    '''
    print(f"Il col_names_dict di questo tensore è: \n{tensor_frame.col_names_dict}\n\n")

    #### stype_encoder_dict ####
    '''
      L'ultimo parametro da passare è un dizionario che serve all'encoder per "decidere" quale encoder utilizzare per ogni tipo di dato (stype).
      Nel nostro caso abbiamo:
      stype_encoder_dict = {
                stype.categorical: EmbeddingEncoder(),
                stype.numerical: LinearEncoder(),
                stype.embedding: LinearEmbeddingEncoder(),
                stype.timestamp: TimestampEncoder()
            }
      Quindi, l'encoder utilizzerà:
      - EmbeddingEncoder per i dati categorici,
      - LinearEncoder per i dati numerici,
      - LinearEmbeddingEncoder per i dati embedding,
      - TimestampEncoder per i dati temporali.
    '''
    print(f"Il nostro stype_encoder_dict è:\n{stype_encoder_dict}")
    break

Il TensorFrame incontrato ha un elemento di tipo categorico al suo interno: 
TensorFrame(
  num_cols=5,
  num_rows=1,
  categorical (1): ['year'],
  numerical (1): ['round'],
  timestamp (2): ['date', 'time'],
  embedding (1): ['name'],
  has_target=False,
  device='cpu',
)


Il numero di colonne di questo TensorFrame è: 5, quindi out_channels sarà uguale a 5


Questo TensorFrame è di tipo: races
Le statistiche di questo TensorFrame sono: 
{'year': {<StatType.COUNT: 'COUNT'>: ([2023, 2022, 2021, 2019, 2018, 2016, 2017, 2012, 2015, 2014, 2013, 2011, 2010, 2005, 2008, 2004, 2006, 2007, 2001, 1995, 1997, 2000, 1977, 2002, 2020, 2009, 2003, 1999, 1998, 1988, 1996, 1994, 1993, 1992, 1991, 1990, 1989, 1987, 1978, 1985, 1984, 1982, 1986, 1976, 1983, 1981, 1979, 1974, 1973, 1980, 1975, 1970, 1968, 1972, 1958, 1967, 1971, 1969, 1965, 1964, 1963, 1960, 1966, 1962, 1959, 1954, 1953, 1952, 1951, 1956, 1957, 1961, 1955, 1950], [22, 22, 22, 21, 21, 21, 20, 20, 19, 19, 19, 19, 19, 19, 18, 18, 18, 17,